In [2]:
import pandas as pd
import numpy as np
import jieba

```
df = pd.read_csv('final.csv')
df=df[["alblum",'author','content','titles']]
df.columns = ['album', 'singer','lyrics','title']
df.to_csv('final_df.csv',encoding='utf-8')

data = pd.read_csv('final_df.csv')
del data['Unnamed: 0']
data.head()
data.describe()






```

In [9]:
with open('stop_words.txt',encoding="utf8") as f:
    stop_words = f.read().splitlines() #载入stopword字典

In [10]:
def jieba_lyric(item):
    word_list = ""
    after = ",".join(jieba.cut(item))
    new_list = after.split(",")
    for word in new_list:
        if word not in stop_words and len(word)>0:
            word_list += word
    return word_list #不返回唯一值

In [11]:
import re
def title(item):
    item = item.replace(",", "")
    return re.findall(r"[\w']+", item)[0]

```
def generate_df(singer_name,data):
    df = data[data["singer"] == singer_name]
    df = df[pd.notnull(df['lyrics'])]
    unique_df = df.drop_duplicates(['lyrics'], keep='last')
    unique_df = unique_df[unique_df["lyrics"] != "Sorry，该歌曲暂无歌词。"]
    unique_df.head()
    unique_df= unique_df.reset_index(drop=True)
    unique_df["lyrics"] = unique_df["lyrics"].apply(jieba_lyric)
    unique_df["title"] = unique_df["title"].apply(title)
    unique_df = unique_df.drop_duplicates(['title'], keep='last')
    return unique_df
df_lee = generate_df("王力宏",data)
df_jay = generate_df("周杰伦",data)
df_lin = generate_df("林俊杰",data)
df_pan = generate_df("潘玮柏",data)
result = pd.concat([df_lee,df_jay,df_lin,df_pan])
result.to_csv("alltogether.csv",encoding='utf-8',index=False)
```

In [3]:
df = pd.read_csv('alltogether.csv')
df = df[["album","singer","lyrics","title"]]
df.head()

,album,singer,lyrics,title
0,《爱程集》,王力宏,比你更好 如什么都可修补 孤单真可起诉 或者只得我欠回报 谁想将恋爱当回报 别的手 ...,比你更好
1,《爱情攻略》,王力宏,想爱 就是这一夜 凝结了空气和时间 强烈感到你就在身边 我已明白那就是爱 沉溺幸福的味道叫...,想爱
2,《公转自转》,王力宏,One of these days 我的脚很轻 走在台北 一个人 半夜的自己 空荡的...,ONE
3,《公转自转》,王力宏,Please come back to me 邬裕康 盏盏街灯後退 车里播放思念 ...,Please
4,《盖世英雄演唱会》,王力宏,Follow Me Live 词曲 你我之间距离不是 唯一的联系 你我之间还有一种 跳动的...,Follow


```
sum = 0
for i in result["lyrics"]:
    try:
        sum = sum + len(i)
    except:
        pass
```

## 歌曲分词

## 情感分析

In [21]:
from bosonnlp import BosonNLP
nlp = BosonNLP('3Txfyn4T.14799.EyO6wjbCihaV')

In [22]:
def sentiment(sentence):
    sentence = sentence[15:]
    return nlp.sentiment(sentence,model='weibo')

In [23]:
def analysis_emotion(df):
    df["sentiment"] = df["lyrics"].apply(sentiment)
    df["positive"] = [i[0][0] for i in df["sentiment"]]
    df["negative"] = [i[0][1] for i in df["sentiment"]]
    positive_score = 0
    negative_score = 0
    for i in df["sentiment"]:
        positive_score += i[0][0]
        negative_score += i[0][1]
    
    return df,positive_score,negative_score

## 王力宏

In [15]:
df_lee = df[df["singer"] == "王力宏"]
df_lee, positive_score_1, nagetive_score_1 = analysis_emotion(df_lee)

F:\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
F:\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
F:\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-do

In [16]:
print(positive_score_1, nagetive_score_1)
df_lee.sort_values("positive",ascending=False).head(20)
#df_lee.sort_values("negative",ascending=False).head(30)

77.1678923337349 76.83210766626512


,album,singer,lyrics,title,sentiment,positive,negative
49,《畅爽开怀》,王力宏,畅爽开怀 崔惟楷 Andrew Bloch Ed Dunne Butch Walker Ce...,畅爽开怀,"[[0.9981289389133092, 0.001871061086690763]]",0.998129,0.001871
80,《Sony Music-Man 世界巡回演唱会》,王力宏,放开你的心 baby我爱你快乐永远来得及 so come on now and放开你的心 ...,放开你的心,"[[0.9876177314093767, 0.012382268590623321]]",0.987618,0.012382
138,《火力全开》,王力宏,依然爱你 一闪一闪亮晶晶 留下岁月的痕迹 我的世界的中心 依然还是你 一年一年又一年 飞逝...,依然爱你,"[[0.9861348835402827, 0.013865116459717372]]",0.986135,0.013865
126,《16个夏天 电视原声带》,王力宏,爱你等于爱自己 淋我淋过的雨 吹你吹过的风 比风雨还亲密 两个人的影子 在同一个天地 ...,爱你等于爱自己,"[[0.9788506127022041, 0.021149387297795925]]",0.978851,0.021149
48,《永远的第一天》,王力宏,欢喜城 谢铭祜 There is a city 要什么都有 热情的地方 交通很拥挤 遇到...,欢喜城,"[[0.9786844713968051, 0.02131552860319487]]",0.978684,0.021316
145,《大马 Music Man 纪念精选辑》,王力宏,另一个天堂 葛大为David K. 张靓颖 你取代 前一秒我生命的空白 问题忽然找到...,另一个天堂,"[[0.9778085385988959, 0.022191461401104045]]",0.977809,0.022191
110,《改变自己》,王力宏,爱的鼓励 Hey 太阳会出现 只要给一次机会 心情一飞冲天 带你的烦恼Away 每一次...,爱的鼓励,"[[0.9777272029915178, 0.022272797008482205]]",0.977727,0.022273
14,《Sony Music-Man 世界巡回演唱会》,王力宏,爱你等于爱自己 淋我淋过的雨 吹你吹过的风 比风雨还亲蜜 两个人的影子 在同一个天地 拥有一...,爱你等如爱自己,"[[0.9612962291465775, 0.038703770853422456]]",0.961296,0.038704
9,《不可能错过你》,王力宏,钓灵感 多喝水 多喝水 风扇吹 风扇吹 眼睛垂 眼睛垂 我假装小睡 我假装小睡 灵感游去...,钓灵感,"[[0.9543137225675283, 0.045686277432471646]]",0.954314,0.045686
22,《好想你》,王力宏,爱依然执着 陈乐融 Enrique LglesiasRoberto Morales 在一见...,爱依然执着,"[[0.9528981142280897, 0.0471018857719103]]",0.952898,0.047102


## 周杰伦

In [17]:
df_jay = df[df["singer"] == "周杰伦"]
df_jay,positive_score_2, nagetive_score_2 = analysis_emotion(df_jay)

F:\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
F:\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
F:\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-do

In [18]:
print(positive_score_2, nagetive_score_2)
df_jay.sort_values("negative",ascending=True).head(30)
df_jay.sort_values("negative",ascending=False).head(30)

79.83191082097741 98.16808917902253


,album,singer,lyrics,title,sentiment,positive,negative
231,《周杰伦 2004 无与伦比 演唱会 Live CD》,周杰伦,倒带 方文山 我受够了等待 你所谓的安排 到底多久多久才来 你总是要我乖 慢慢计划...,倒带,"[[0.0027401376558924273, 0.9972598623441076]]",0.002740,0.997260
243,《跨时代》,周杰伦,说了再见 古小力黄淩嘉 天凉了 雨下了 你走了 清楚了 我爱的 遗失了 落叶飘在湖面上睡...,说了再见,"[[0.004703255345954949, 0.995296744654045]]",0.004703,0.995297
258,《八度空间》,周杰伦,半岛铁盒 词曲 读者呀 小姐请问一下是没有卖半岛铁盒 店员有呀你从前面右转的第二排架上就有...,半岛铁盒,"[[0.010384226207117653, 0.9896157737928823]]",0.010384,0.989616
306,《周杰伦的床边故事》,周杰伦,爱情废柴 词曲 圣诞节 剩下单人的剩单节 过条街 最好又给我下起雪 这么衰 这么刚好 这么...,爱情废柴,"[[0.01093087670267967, 0.9890691232973203]]",0.010931,0.989069
276,《八度空间》,周杰伦,最后的战役 方文山 机枪扫射声中我们寻找遮蔽的战壕 儿时沙雕的城堡毁坏了重新盖就好 可...,最后的战役,"[[0.011781353257566352, 0.9882186467424336]]",0.011781,0.988219
286,《范特西》,周杰伦,安静 词曲 只剩下钢琴陪我谈了一天 睡著的大提琴 安静的旧旧的 我想你已表现的非常明白 我...,安静,"[[0.01430202051911289, 0.9856979794808871]]",0.014302,0.985698
284,《我很忙》,周杰伦,彩虹 哪里有彩虹告诉我 能不能把我的愿望还给我 为什么天这么安静 所有的云都跑到我这里 有...,彩虹,"[[0.021275716762857888, 0.9787242832371421]]",0.021276,0.978724
301,《周杰伦的床边故事》,周杰伦,一点点 方文山 几次争辩 几次的失眠 而我却 视而不见 从不 先说抱歉 工作优先 跟朋友...,一点点,"[[0.03299101632265211, 0.9670089836773479]]",0.032991,0.967009
325,《钢琴恋曲101》,周杰伦,龙卷风 爱像一阵风 吹完它就走 这样的节奏 谁都无可奈何 没有你以后 我灵魂失控 黑云在降落...,龙卷风,"[[0.03692975447005431, 0.9630702455299457]]",0.036930,0.963070
190,《周杰伦同名专辑》,周杰伦,斗牛 方文山 有什么不妥 有话就直说 别窝在角落 不爽就反驳 到底拽什么 懂不懂篮球 有...,斗牛,"[[0.04271132131158095, 0.957288678688419]]",0.042711,0.957289


## 林俊杰

In [19]:
df_lin = df[df["singer"] == "林俊杰"]
df_lin,positive_score_3, nagetive_score_3 = analysis_emotion(df_lin)

F:\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
F:\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
F:\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-do

In [20]:
print(positive_score_3, nagetive_score_3)
df_lin.sort_values("negative",ascending=True).head(20)
df_lin.sort_values("negative",ascending=False).head(20)

56.23126097560978 50.76873902439022


,album,singer,lyrics,title,sentiment,positive,negative
400,《西界》,林俊杰,大男人小女孩 不是我的错 我们都听过 完美的时候 要更多 吵架时候说的太多 有时候的...,大男人,"[[0.015449980903286753, 0.9845500190967132]]",0.015450,0.984550
344,《乐行者》,林俊杰,压力 张思尔 它偷偷摸摸 混进了人类群中 话没有多说 打开了门的锁 我们的生活 忙得我们...,压力,"[[0.02242528882948369, 0.9775747111705163]]",0.022425,0.977575
386,《编号89757》,林俊杰,木乃伊 林秋离简胜 你说要我 死了这颗心 最好永远 存放在埃及 像木乃伊 保持着神秘 敲...,木乃伊,"[[0.02415136296196596, 0.975848637038034]]",0.024151,0.975849
367,《西界》,林俊杰,无聊 林秋离 没事做就会无聊 没有地方动手动脚 没事做就会无聊 没有地方动手动脚 闷到就...,无聊,"[[0.030917006996304952, 0.969082993003695]]",0.030917,0.969083
392,《乐行者》,林俊杰,会有那么一天 张思尔 一九四三 世界大战 阿嬷年轻的时候 爷爷爱他那么多 他们感情很深 ...,会有那么一天,"[[0.0379411530166881, 0.9620588469833119]]",0.037941,0.962059
403,《她说 概念自选辑》,林俊杰,我很想爱他 何启弘 天空 下起雨了 他撑的伞 在你身边陪着 可是 我不快乐 因为看见 ...,我很想爱他,"[[0.041873167635545494, 0.9581268323644545]]",0.041873,0.958127
431,《乌鸦嘴妙女郎 电视原声带》,林俊杰,不潮不用花钱 林怡凤RAPJJ Hey greedydont fret What yo...,不潮不用花钱,"[[0.043144112336047535, 0.9568558876639525]]",0.043144,0.956856
356,《曹操》,林俊杰,Now That Shes Gone Girl you know i miss you s...,Now,"[[0.04752591111190041, 0.9524740888880996]]",0.047526,0.952474
359,《乐行者》,林俊杰,会读书 灯 不能熄灭 熬过今夜 就能从书中逃回到 这个世界 我 我不明白 这个年代 怎么还...,会读书,"[[0.05308425053416821, 0.9469157494658318]]",0.053084,0.946916
384,《乐行者》,林俊杰,不懂 己经好远了 退也有一点累了 我们都不知道路多远 走到何时才歇一歇 不如就现在吧 让我...,不懂,"[[0.053790769360757884, 0.9462092306392421]]",0.053791,0.946209


## 潘玮柏

In [25]:
df_pan = df[df["singer"] == "潘玮柏"]
df_pan,positive_score_4, nagetive_score_4 = analysis_emotion(df_pan)

F:\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
F:\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
F:\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-do

In [26]:
print(positive_score_4, nagetive_score_4)
df_pan.sort_values("negative",ascending=True).head(20)
#df_pan.sort_values("negative",ascending=False).head(20)

53.759669715265055 52.240330284734945


,album,singer,lyrics,title,sentiment,positive,negative
541,《零零七》,潘玮柏,亲爱的 我 叮咚的 弹奏着 故事里那首儿歌 我 叮咚的 弹奏着 故事里那首儿歌 我 叮咚的...,亲爱的,"[[0.9911040483469683, 0.008895951653031748]]",0.991104,0.008896
462,《我的麦克风》,潘玮柏,歌曲I Like You Like That 歌手 你的微笑很无邪 你会坚定的拒绝 你依赖直...,I,"[[0.9860735737404903, 0.013926426259509716]]",0.986074,0.013926
486,《24个比利》,潘玮柏,专属于你 李念和 Cause youre my baby my baby my baby ...,专属于你,"[[0.9840065641117866, 0.015993435888213427]]",0.984007,0.015993
452,《零零七》,潘玮柏,爱的歌 梁思成 梁思成 蔡庭贵 关于爱的故事太多 我们只是其中一个 关于爱的故事太多...,爱的歌,"[[0.9647464265463013, 0.03525357345369872]]",0.964746,0.035254
514,《零零七》,潘玮柏,限量发行 我是限量发行 我是限量发行 没有代替品 我是限量发行 没有代替品 我是限量发行...,限量发行,"[[0.9585737753125295, 0.041426224687470516]]",0.958574,0.041426
538,《反转地球》,潘玮柏,反转地球 Bow bow bow 让我看到你双手 对抗地心引力一起反转地球 Bow bow...,反转地球,"[[0.9433123950067748, 0.056687604993225275]]",0.943312,0.056688
479,《808》,潘玮柏,次世代 Na Na Na Na Na Na Na Na Hey Hey Hey Goo...,次世代,"[[0.9431633797581891, 0.056836620241810853]]",0.943163,0.056837
449,《壁虎漫步》,潘玮柏,Good Love 易家扬 Perry Lee Seung Ho Song Back Ky...,Good,"[[0.9425676091397137, 0.05743239086028634]]",0.942568,0.057432
444,《玩酷》,潘玮柏,曲目爱不离 PS送给我最爱的宝贝群 嘿 一定就是你 缘份天造地设 完美的相遇 陌生人海里 ...,爱不离,"[[0.9282926204267207, 0.07170737957327931]]",0.928293,0.071707
466,《Baller》,潘玮柏,跟我走吧 徐世珍 这支手机跟你说过话 这杯子你喝过茶 这个月亮你也在看着吗 还有温度留在...,跟我走吧,"[[0.9178476310017616, 0.08215236899823837]]",0.917848,0.082152


In [28]:
result = pd.concat([df_lee,df_jay,df_lin,df_pan])
result.to_csv("dataforserach.csv",encoding="utf_8_sig")

In [90]:
#单词分析
def word_analysis(df):
    word_list ={}
    for i in df["lyrics"]:
        lyric_list = []
        after = ",".join(jieba.cut(i))
        lyric_list = after.split(",")
        for word in lyric_list:
            if word not in stop_words and len(word)>0:
                if word not in word_list:
                    word_list[word] = 1
                else:
                    word_list[word] += 1

    count = [(word,word_list[word]) for word in word_list]
    count.sort(key=lambda x: x[1],reverse=True)
    return count
#('\u3000', 167) 中文全角空格

In [94]:
jay_count = word_analysis(df_jay)

In [93]:
def find_season(count_list):
    season = [0,0,0,0]
    for i in count_list:
        if i[0].find("春") >=0:
            print (i)
            season[0] += i[1]
        if i[0].find("夏")>=0:
            print (i)
            season[1] += i[1]
        if i[0].find("秋")>=0:
            print (i)
            season[2] += i[1]
        if i[0].find("冬")>=0:
            print (i)
            season[3] += i[1]
    return season

In [98]:
find_season(word_analysis(df_pan))

('秋冬', 3)
('秋冬', 3)
('春夏', 3)
('春夏', 3)
('青春', 1)
('夏娃', 1)
('夏天', 1)
('春风', 1)


[5, 5, 3, 3]

In [101]:
def find_his(count_list):
    season = [0,0,0,0]
    for i in count_list:
        if i[0].find("你") >=0:
            print (i)
            season[0] += i[1]
        if i[0].find("我")>=0:
            print (i)
            season[1] += i[1]
        if i[0].find("他")>=0:
            print (i)
            season[2] += i[1]
        if i[0].find("她")>=0:
            print (i)
            season[3] += i[1]
    return season

In [120]:
len(word_analysis(df_pan))

5384

In [121]:
def count_total(count_list):
    total = 0
    for i in count_list:
        total += i[1]
    return total

In [125]:
count_total(word_analysis(df_lin))

33277

In [114]:
def love(count_list):
    season = 0
    for i in count_list:
        if i[0].find("爱") >=0:
            season += i[1]
    return season

In [119]:
love(word_analysis(df_pan))

601